In [1]:
from functools import partial
import inspect
import sys
import numpy as np
import tensorflow as tf

import tensorcircuit as tc

2025-02-20 10:35:49.974359: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-20 10:35:52.249350: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-20 10:35:53.459614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740069353.900056   45001 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740069354.188833   45001 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-20 10:35:59.869733: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
def get_circuit(n):
    c = tc.Circuit(n)  # initialize a circuit object with n qubits
    for i in range(n):
        c.H(i)  # apply Hadamard gate on each qubit
    c.cnot(0, 1)  # apply cnot with control qubit on 0-th qubit
    c.CNOT(n - 1, n - 2)  # capitalized API also works
    return c

In [3]:
# the corresponding matrix for these gates definition
for g in tc.Circuit.sgates:
    gf = getattr(tc.gates, g)
    print(g)
    print(tc.gates.matrix_for_gate(gf()))

i
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
x
[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]
y
[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]
z
[[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]
h
[[ 0.70710677+0.j  0.70710677+0.j]
 [ 0.70710677+0.j -0.70710677+0.j]]
t
[[1.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.70710677+0.70710677j]]
s
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+1.j]]
td
[[1.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.70710677-0.70710677j]]
sd
[[1.+0.j 0.+0.j]
 [0.+0.j 0.-1.j]]
wroot
[[ 0.70710677+0.j  -0.5       -0.5j]
 [ 0.5       -0.5j  0.70710677+0.j ]]
cnot
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]]
cz
[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]]
swap
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]
cy
[[1.+0.j 

In [4]:
c = get_circuit(3)
ir = c.to_qir()  # intermediate representation of the circuit
ir

[{'gatef': h,
  'gate': Gate(
      name: 'h',
      tensor:
          array([[ 0.70710677+0.j,  0.70710677+0.j],
                 [ 0.70710677+0.j, -0.70710677+0.j]], dtype=complex64),
      edges: [
          Edge('cnot'[2] -> 'h'[0] ),
          Edge('h'[1] -> 'qb-0'[0] )
      ]),
  'index': (0,),
  'name': 'h',
  'split': None,
  'mpo': False},
 {'gatef': h,
  'gate': Gate(
      name: 'h',
      tensor:
          array([[ 0.70710677+0.j,  0.70710677+0.j],
                 [ 0.70710677+0.j, -0.70710677+0.j]], dtype=complex64),
      edges: [
          Edge('cnot'[3] -> 'h'[0] ),
          Edge('h'[1] -> 'qb-1'[0] )
      ]),
  'index': (1,),
  'name': 'h',
  'split': None,
  'mpo': False},
 {'gatef': h,
  'gate': Gate(
      name: 'h',
      tensor:
          array([[ 0.70710677+0.j,  0.70710677+0.j],
                 [ 0.70710677+0.j, -0.70710677+0.j]], dtype=complex64),
      edges: [
          Edge('cnot'[2] -> 'h'[0] ),
          Edge('h'[1] -> 'qb-2'[0] )
      ]),
  'index':

In [5]:
ir[0]["gatef"]().tensor, ir[-1]["gate"].tensor  # the actually unitary for each gate

(array([[ 0.70710677+0.j,  0.70710677+0.j],
        [ 0.70710677+0.j, -0.70710677+0.j]], dtype=complex64),
 array([[[[1.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j]],
 
         [[0.+0.j, 1.+0.j],
          [0.+0.j, 0.+0.j]]],
 
 
        [[[0.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j]],
 
         [[0.+0.j, 0.+0.j],
          [1.+0.j, 0.+0.j]]]], dtype=complex64))

In [6]:
# compute the final output quantum state from the circuit
c.state()

array([0.35355335+0.j, 0.35355335+0.j, 0.35355335+0.j, 0.35355335+0.j,
       0.35355335+0.j, 0.35355335+0.j, 0.35355335+0.j, 0.35355335+0.j],
      dtype=complex64)

In [7]:
# compute some expectation values, say <X1>
x1 = c.expectation([tc.gates.x(), [1]])

# or <Z1Z2>
z1z2 = c.expectation([tc.gates.z(), [1]], [tc.gates.z(), [2]])

print(x1, z1z2)

(0.9999998+0j) 0j


In [8]:
# make some random samples
for _ in range(10):
    print(c.perfect_sampling())

(array([0., 0., 0.], dtype=float32), np.float32(0.12499998))
(array([0., 0., 0.], dtype=float32), np.float32(0.12499998))
(array([1., 0., 1.], dtype=float32), np.float32(0.12499998))
(array([0., 0., 0.], dtype=float32), np.float32(0.12499998))
(array([1., 1., 1.], dtype=float32), np.float32(0.12500021))
(array([0., 0., 1.], dtype=float32), np.float32(0.12499998))
(array([1., 1., 1.], dtype=float32), np.float32(0.12500021))
(array([1., 1., 0.], dtype=float32), np.float32(0.12499998))
(array([0., 0., 1.], dtype=float32), np.float32(0.12499998))
(array([0., 0., 1.], dtype=float32), np.float32(0.12499998))


In [1]:
# we can easily switch simulation backends away from NumPy!

with tc.runtime_backend("tensorflow") as K:
    c = get_circuit(3)
    print(c.state())

with tc.runtime_backend("jax") as K:
    c = get_circuit(3)
    print(c.state())

with tc.runtime_backend("pytorch") as K:
    # best performance and full functionality are not guaranteed on pytorch backend
    c = get_circuit(3)
    print(c.state())

NameError: name 'tc' is not defined

In [ ]:
# circuit gates that accept parameters

print(tc.Circuit.vgates)

In [ ]:
# see the keyword parameters (with type float) for each type of variable gate
for g in tc.Circuit.vgates:
    print(g, inspect.signature(getattr(tc.gates, g).f))